# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [1]:
%pip install -q anywidget ipywidgets "wiki3_ai>=0.5.1"

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [5]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [6]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [7]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [8]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Clean syntax flows free,
Logic blooms, a swift design,
Code with grace unfolds. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [9]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

Replace `'your_file.csv'` with the actual filename.  `csv.reader` iterates over each row of the CSV.



In [13]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """
  Reads a CSV file and prints each row.

  Args:
    filename: The name of the CSV file to read.
  """
  try:
    # Open the CSV file in read mode ('r'). The 'with' statement ensures the file is automatically closed.
    with open(filename, 'r') as file:
      # Create a CSV reader object. This object allows you to iterate over the rows of the CSV file.
      reader = csv.reader(file)

      # Iterate over each row in the CSV file. Each 'row' is a list of strings, representing the values in each column.
      for row in reader:
        # Print the row to the console. You can replace this with your desired processing logic.
        print(row)
  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")
```


## Streaming Response

Stream the response as it's generated.

In [10]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Why was the computer cold? 

It left its Windows open! 



In [11]:
async for chunk in assistant_session.prompt_streaming("Read me a long poem please."):
    print(chunk, end="", flush=True)
print()

Okay, here's a longer poem for you. It's a slightly whimsical piece about finding beauty in unexpected places.

**Ephemeral Echoes**

The dawn arrives, a blush of rose and gray,
Painting the slumbering world in gentle sway.
A spider's web, a silver, fragile lace,
Hangs glistening, a masterpiece in place.

Not grand cathedral, nor a sculpted stone,
But delicate threads, exquisitely sown.
A fleeting marvel, born of dew-kissed night,
A whispered echo of the fading light.

The city wakes, a rumble and a hum,
Steel giants stir, their tireless work begun.
A concrete jungle, stark and cold and vast,
Yet tiny blossoms push through cracks so fast.

A splash of crimson, a vibrant, hopeful hue,
A testament to life, forever new.
A stubborn dandelion, bold and bright,
Defying pavement, bathed in morning light.

The wind it sighs, a mournful, ancient tune,
Rustling leaves beneath the harvest moon.
Each fallen leaf, a story softly told,
Of seasons turning, brave and strong and bold.

And in the quiet

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [12]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

Assistant: A list comprehension is a concise way to create lists in Python. It's essentially a shorthand for creating a new list based on an existing iterable.

Here's the basic syntax:

```python
new_list = [expression for item in iterable if condition]
```

*   **expression:** What you want to put in the new list.
*   **item:**  A variable representing each element in the iterable.
*   **iterable:**  A sequence (like a list, tuple, string, range) to iterate over.
*   **condition (optional):** A filter to include only certain items.

**Example:**

```python
numbers = [1, 2, 3, 4, 5]
squared_numbers = [x**2 for x in numbers]  # [1, 4, 9, 16, 25]

even_squares = [x**2 for x in numbers if x % 2 == 0] # [4, 16]
```

Essentially, it's a compact `for` loop combined with an `if` statement for filtering. They are often faster and more readable than traditional `for` loops for creating lists.



In [13]:
# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: ```python
# Create a list of numbers
numbers = [1, 2, 3, 4, 5]

# Double each number using list comprehension
doubled_numbers = [x * 2 for x in numbers]

print(doubled_numbers)  # Output: [2, 4, 6, 8, 10]


# Create a list of even numbers from the original list
even_numbers = [x for x in numbers if x % 2 == 0]

print(even_numbers)  # Output: [2, 4]

# Convert strings to uppercase
words = ["hello", "world", "python"]
uppercase_words = [word.upper() for word in words]

print(uppercase_words) # Output: ['HELLO', 'WORLD', 'PYTHON']
```

The examples demonstrate how list comprehensions create new lists based on existing iterables, applying transformations (doubling, converting to uppercase) or filtering elements (selecting even numbers). They're a powerful and elegant feature of Python.


## Check Token Usage

Monitor how many tokens you've used.

In [14]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 1051/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [15]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [17]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

print("=== Branch 1 ===")
async for chunk in branch1.prompt_streaming("The dragon was friendly and helpful."):
    print(chunk, end="", flush=True)
print("=== End of 1 ===")

print("=== Branch 2 ===")
async for chunk in branch2.prompt_streaming("The dragon was fierce and terrifying."):
    print(chunk, end="", flush=True)
print("=== End of 2 ===")

=== Branch 1 ===
Once upon a time, there was a dragon named Zephyr. But unlike the fiery behemoths of legend, Zephyr wasn’t known for scorching villages or hoarding gold. Zephyr was known… for collecting lost things, and for his remarkably kind heart.

His scales weren’t shimmering emerald or ruby, but a muted, dusty lavender, the color of twilight. He didn't breathe fire, he breathed gentle breezes that carried the scent of lavender and forgotten memories. He resided not in a volcanic mountain, but in a hidden valley nestled between whispering willow trees and a babbling brook.

Zephyr's hoard wasn't gold, but a breathtaking collection of lost objects. A single, mismatched sock, a tarnished silver locket containing a faded picture of a laughing child, a chipped porcelain doll missing an arm, a compass that pointed not North, but towards hope. Each item hummed with a silent story, a whisper of a life once lived.

He wasn’t just a collector; he was incredibly helpful.  If a villager los

## Cleanup

Destroy sessions when you're done to free up resources.

In [18]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
